In [ ]:
import pandas as pd
import numpy as np

In [ ]:
counties = pd.read_csv('data/counties.csv')
manufacturers = pd.read_csv('data/manufacturers.csv')
products = pd.read_csv('data/products.csv')
retailers = pd.read_csv('data/retailers.csv')
stores = pd.read_csv('data/stores.csv')

manufacturers['ratio'] = manufacturers.revenue / manufacturers.revenue.sum()
retailers['ratio'] = retailers.revenue / retailers.revenue.sum()
counties['ratio'] = counties.population / counties.population.sum()

In [ ]:
from datetime import date, timedelta, datetime
import math
import random
import time

start_date = date(2016, 1, 1)
end_date = date(2017, 12, 31)
curr_date = start_date

delta = timedelta(days=1)

sales_per_day = 1000

sale_id = 1
d = []
start = time.time()
ii = 0
while curr_date <= end_date:
    for _, man in manufacturers.iterrows():
        manufacturer_add_date = datetime.strptime(man.add_date, '%Y-%m-%d %H:%M:%S').date()
        if manufacturer_add_date > curr_date:
            break
        for _, ret in retailers.iterrows():
            for _, cou in counties.iterrows():
                sales_percent = ret.ratio * cou.ratio * man.ratio
                num_sales = math.floor(sales_per_day * sales_percent * np.random.normal(1, 0.1))
                for n in range(num_sales):
                    product_id = random.choice(products.product_id)
                    unit_price = round(np.random.normal(10, 2), 2)
                    qty = math.ceil(np.random.exponential(2))
                    date_formatted = curr_date.strftime("%Y-%m-%d 00:00:00")
                    store_id = int(stores[(stores.retailer_id == ret.retailer_id) & (stores.county_id == cou.county_id)]['store_id'])
                    d.append(
                        {
                            'sale_id': sale_id,
                            'store_id': store_id,
                            'product_id': product_id,
                            'date': date_formatted,
                            'unit_price': unit_price,
                            'qty': qty 
                        }
                    )
                    sale_id += 1
    curr_date += delta
    ii += 1
    if (ii % 100 == 0):
        print(ii, time.time() - start)
df = pd.DataFrame(d)

In [ ]:
df.to_csv('sales.csv', index=False)